In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); 
using ForwardDiff
using Test
using RobotZoo
using RobotDynamics
using LinearAlgebra
using StaticArrays
using SparseArrays
using BlockArrays

include("../src/nlp.jl")
include("../src/qp.jl")
include("../src/sqp.jl")

 Activating environment at `~/Classes/OptimalControl/hw3_solutions/Project.toml`
┌ Info: Precompiling RobotZoo [74be38bb-dcc2-4b9e-baf3-d6373cd95f10]
└ @ Base loading.jl:1278


solve_qp! (generic function with 2 methods)

# The Problem

In [32]:
# Build the problem
model = RobotZoo.Cartpole()
n,m = size(model)
T = 101
tf = 2.0
dt = tf / (T-1)

# Initial & final condition
x0 = @SVector zeros(n)
xf = SA[0,pi,0,0];

# Cost function
Q = Diagonal(fill(1e-2,n))
R = Diagonal(fill(1e-1,m))
Qf = Diagonal(fill(1e1,n))
costfun = LQRCost(Q,R,xf)
costterm = LQRCost(Qf,R,xf)
obj = push!(fill(costfun,T-1), costterm)

# Initial Guess (linear interpolation)
X = [x0 + (xf - x0)*t for t in range(0,1, length=T)]
U = [@SVector zeros(m) for k = 1:T-1];

# Solve w/ SQP

In [33]:
# NLP
nlp = NLP(model, obj, tf, T, x0, xf)
Z = packZ(nlp, X, U)
λ = zeros(num_duals(nlp));

In [34]:
# Solve w/ SQP
Zstar, λstar = solve_sqp!(nlp, Z, λ, verbose=1, iters=200, gn=false, eps_primal=1e-4);

Iteration 1: cost = 1.67, res_p = 1.83e-01, res_d = 2.93e+00,   α = 1.00, ΔJ: -6.96e+02, Δϕ: 9.31e+02, reg: 1.00e-06, pen: 523, soc: 0
Iteration 2: cost = 697.64, res_p = 3.68e+02, res_d = 3.33e+00,   α = 1.00, ΔJ: 3.26e+02, Δϕ: 1.36e+04, reg: 1.00e-06, pen: 523, soc: 0
Iteration 3: cost = 371.15, res_p = 1.38e+02, res_d = 3.76e-01,   α = 0.50, ΔJ: 1.20e+02, Δϕ: 2.47e+02, reg: 1.00e-06, pen: 523, soc: 0
Iteration 4: cost = 251.18, res_p = 8.06e+01, res_d = 3.51e-01,   α = 0.13, ΔJ: 4.16e+01, Δϕ: 8.96e+01, reg: 1.00e-06, pen: 523, soc: 0
Iteration 5: cost = 209.62, res_p = 6.95e+01, res_d = 3.44e-01,   α = 0.50, ΔJ: 5.17e+01, Δϕ: 3.39e+02, reg: 1.00e-06, pen: 523, soc: 0
Iteration 6: cost = 157.92, res_p = 2.98e+01, res_d = 2.98e-01,   α = 1.00, ΔJ: -1.29e+01, Δϕ: 6.27e+02, reg: 1.00e-06, pen: 523, soc: 1
Iteration 7: cost = 170.87, res_p = 3.44e+01, res_d = 7.94e-02,   α = 1.00, ΔJ: 9.61e+00, Δϕ: 1.30e+02, reg: 1.00e-06, pen: 523, soc: 1
Iteration 8: cost = 161.26, res_p = 1.80e+01, re

# Visualization

In [18]:
include("cartpole.jl")

visualize! (generic function with 2 methods)

In [19]:
vis = Visualizer()
set_mesh!(vis, model)
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/brian/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("rod" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x72, 0x6f, 0x64], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"cart" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))),"box" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x62, 0x6f, 0x78], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()),"hinge" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65], nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()))))))))), Set{Any}(), ip"127.0.0.1", 8702))

In [35]:
Xstar = [Z[xi] for xi in nlp.xinds]
visualize!(vis, model, tf, Xstar)